<a href="https://colab.research.google.com/github/Mark-Soucek/CHE4061_Fall2025/blob/main/PonchonSavarit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget -N -q https://raw.githubusercontent.com/chetools/chetools/main/tools/che5.ipynb -O che5.ipynb
!pip install importnb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.5 MB/s eta 0:00:00


In [ ]:
from importnb import Notebook
with Notebook():
    from che5 import Props

import numpy as np
import scipy as sp
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = "plotly_dark"
import jax
import jax.numpy as jnp
jax.config.update("jax_enable_x64", True)
eps =  np.finfo(np.float64).eps

#Ponchon Savarit Calculations

In [ ]:
#Vn1*Hvn1 = Ln*Hln + D*Hd + Qc
#(Ln + D)* Hvn1 = Ln * Hln + D*Hd + Qc
#Ln(Hvn1 - Hl) = D*Hd + Qc - D*Hvn1


#Ln/D *(Hvn1 - Hln) = Hd + Qc/D - Hvn1
#Ln/D*(yn1 - xn) = xd - yn1

In [ ]:
p = Props(['Isopropanol','Water'])
F = 1.
z = 0.3
q=1.
Hf = p.Hl(np.array([z, 1-z]), TxAkima(z))

rD = 0.95  #recovery of isopropanol in distillate product
xD = 0.62
D = F*z*rD/xD
B = F - D
xB = F*z*(1-rD)/B
Hb = p.Hl(np.array([xB, 1-xB]), TxAkima(xB))

R=0.5
V1 = D*(R+1)


#Qc/D = V1*(Hv1 - Hd)/D
Hd = p.Hl(np.array([xD, 1-xD]), TxAkima(xD))
Qc = V1*(p.Hv(np.array([xD, 1-xD]), TyAkima(xD))- Hd)
Qb = D*Hd + B*Hb + Qc - F*Hf

In [ ]:
#Ln/D *(Hvn1 - Hln) = Hd + Qc/D - Hvn1
#Ln/D*(yn1 - xn) = xD - yn1
# (Hd + Qc/D - Hvn1)/(Hvn1 - Hln) - (xD - yn1)/(yn1 - xn) = 0
def EMRec(yn1, Hln, xn):
    Hvn1 = p.Hv(np.array([yn1, 1-yn1]), TyAkima(yn1))
    return (Hd + Qc/D - Hvn1)*(yn1 - xn) - (xD - yn1)*(Hvn1 - Hln)

def EMStrip(yn1, Hln, xn):
    Hvn1 = p.Hv(np.array([yn1, 1-yn1]), TyAkima(yn1))
    return (Hln - (Hb - Qb/B))*(yn1-xn) - (xn - xB)*(Hvn1 - Hln)


In [ ]:
yn1s=[xD]
xn1s=[]
yn1=xD
for i in range(100):
    xn1 = x1Akima(yn1).item()
    xn1s.append(xn1)
    if xn1<xB:
        break
    Hl = p.Hl(np.array([xn1, 1-xn1]), TxAkima(xn1))
    print(i,xn1)
    if xn1>z:
        yn1 = sp.optimize.root_scalar(lambda yn1: EMRec(yn1, Hl, xn1), bracket=(xn1,yn1)).root
    else:
        yn1 = sp.optimize.root_scalar(lambda yn1: EMStrip(yn1, Hl, xn1), bracket=(xn1,yn1)).root
    yn1s.append(yn1)


0 0.5674855055564247
1 0.5205547143486929
2 0.46987528188196953
3 0.398466973741908
4 0.2324242332952662
5 0.04670284623297177


In [ ]:
fig=make_subplots()
fig.add_scatter(x=zs, y=Hvs, mode='lines',name='Hvap')
fig.add_scatter(x=zs, y=Hls, mode='lines',name='Hliq')
fig.add_scatter(x=(xD,xD),y=(Hd, Hd+Qc/D), mode='lines', line_color='green')

first_line = False
for (x,y) in zip(xn1s, yn1s):
    fig.add_scatter(x=(x,y),y=(p.Hl(np.array([x, 1-x]), TxAkima(x)), p.Hv(np.array([y, 1-y]), TyAkima(y))), mode='lines', line_dash='dot', line_color='rgb(100, 100, 100)')
    if x>z:
        fig.add_scatter(x=(x,xD),y=(p.Hl(np.array([x, 1-x]), TxAkima(x)), Hd+Qc/D), mode='lines', line_color='green')
    else:
        if first_line == False:
            first_line = True
        else:
            fig.add_scatter(x=(y,xB),y=(p.Hv(np.array([y, 1-y]), TyAkima(y)), Hb-Qb/B), mode='lines', line_color='green')

fig.update_xaxes(range=[0,xD+0.05])
fig.update_layout(width=600, height=600, showlegend=False)